In [2]:
import os, sys, inspect
from pathlib import Path

current_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 

In [3]:
import pandas as pd
import numpy as np
import json
from sklearn import datasets, ensemble, model_selection

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset import RegressionPreset
from evidently.metric_preset import ClassificationPreset
from evidently.metric_preset import TargetDriftPreset

In [71]:
from evidently.metrics import ColumnDriftMetric
from evidently.metrics import DataDriftTable
from evidently.metrics import DatasetDriftMetric
from evidently.metrics import ColumnDistributionMetric
from evidently.metrics import ColumnValuePlot
from evidently.metrics import ColumnQuantileMetric
from evidently.metrics import ColumnCorrelationsMetric
from evidently.metrics import ColumnValueListMetric
from evidently.metrics import ColumnValueRangeMetric
from evidently.metrics import DatasetCorrelationsMetric
from evidently.metrics import ColumnRegExpMetric
from evidently.metrics import ColumnSummaryMetric
from evidently.metrics import ColumnMissingValuesMetric
from evidently.metrics import DatasetSummaryMetric
from evidently.metrics import DatasetMissingValuesMetric

from evidently.metrics import ConflictTargetMetric
from evidently.metrics import ConflictPredictionMetric
from evidently.metrics import TargetByFeaturesTable

In [87]:
from evidently.tests import TestTargetFeaturesCorrelations
from evidently.tests import TestTargetPredictionCorrelation
from evidently.tests import TestCorrelationChanges
from evidently.tests import TestNumberOfDriftedColumns

In [72]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [73]:
#Dataset for Binary Probabilistic Classifcation
bcancer_data = datasets.load_breast_cancer(as_frame='auto')
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

bcancer_label_ref = bcancer_ref.copy(deep=True)
bcancer_label_cur = bcancer_cur.copy(deep=True)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

bcancer_label_ref['prediction'] = model.predict(bcancer_label_ref[bcancer_data.feature_names.tolist()])
bcancer_label_cur['prediction'] = model.predict(bcancer_label_cur[bcancer_data.feature_names.tolist()])

In [74]:
bcancer_ref.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target,prediction
554,12.88,28.92,82.50,514.3,0.08123,0.05824,0.06195,0.02343,0.1566,0.05708,0.2116,1.3600,1.5020,16.830,0.008412,0.021530,0.038980,0.007620,0.01695,0.002801,13.89,35.74,88.84,595.7,0.1227,0.1620,0.2439,0.06493,0.2372,0.07242,1,1.0
427,10.80,21.98,68.79,359.9,0.08801,0.05743,0.03614,0.01404,0.2016,0.05977,0.3077,1.6210,2.2400,20.200,0.006543,0.021480,0.029910,0.010450,0.01844,0.002690,12.76,32.04,83.69,489.5,0.1303,0.1696,0.1927,0.07485,0.2965,0.07662,1,1.0
495,14.87,20.21,96.12,680.9,0.09587,0.08345,0.06824,0.04951,0.1487,0.05748,0.2323,1.6360,1.5960,21.840,0.005415,0.013710,0.021530,0.011830,0.01959,0.001812,16.01,28.48,103.90,783.6,0.1216,0.1388,0.1700,0.10170,0.2369,0.06599,1,1.0
527,12.34,12.27,78.94,468.5,0.09003,0.06307,0.02958,0.02647,0.1689,0.05808,0.1166,0.4957,0.7714,8.955,0.003681,0.009169,0.008732,0.005740,0.01129,0.001366,13.61,19.27,87.22,564.9,0.1292,0.2074,0.1791,0.10700,0.3110,0.07592,1,1.0
208,13.11,22.54,87.02,529.4,0.10020,0.14830,0.08705,0.05102,0.1850,0.07310,0.1931,0.9223,1.4910,15.090,0.005251,0.030410,0.025260,0.008304,0.02514,0.004198,14.55,29.16,99.48,639.3,0.1349,0.4402,0.3162,0.11260,0.4128,0.10760,1,0.8


In [75]:
bcancer_ref.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension',
       'target', 'prediction'],
      dtype='object')

In [76]:
#column_mapping
num_columns = ['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension']
datetime_columns =  None
cat_colummns = None
target_columns = 'target'
prediction_columns = 'prediction'
task = 'classification'

column_mapping = ColumnMapping()
column_mapping.target = target_columns
column_mapping.prediction = prediction_columns
column_mapping.datetime_features = datetime_columns
column_mapping.numerical_features = num_columns
column_mapping.categorical_features = cat_colummns
column_mapping.task = task

In [77]:
column_mapping

ColumnMapping(target='target', prediction='prediction', datetime='datetime', id=None, numerical_features=['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension'], categorical_features=None, datetime_features=None, target_names=None, task='classification', pos_label=1)

## Generate Reports

### 1. Data Quality Reports

In [ ]:
target_drift_tests = Report(metrics=[DataQualityPreset(),])
target_drift_tests.run(current_data=bcancer_ref, reference_data=None, 
                       column_mapping=column_mapping)

In [ ]:
test = target_drift_tests.as_dict()['metrics']

In [27]:
function_name: str = 'ColumnSummaryMetric'
# function_name: str = 'Number of Constant Columns'

In [28]:
missing_cols = []
for result_dict in test:
     if result_dict['metric'] == function_name and \
        result_dict['result']['current_characteristics']['missing_percentage'] >= 0:
            missing_cols.append(str(result_dict['result']['column_name']))

In [30]:
missing_cols

['area error',
 'compactness error',
 'concave points error',
 'concavity error',
 'fractal dimension error',
 'mean area',
 'mean compactness',
 'mean concave points',
 'mean concavity',
 'mean fractal dimension',
 'mean perimeter',
 'mean radius',
 'mean smoothness',
 'mean symmetry',
 'mean texture',
 'perimeter error',
 'radius error',
 'smoothness error',
 'symmetry error',
 'texture error',
 'worst area',
 'worst compactness',
 'worst concave points',
 'worst concavity',
 'worst fractal dimension',
 'worst perimeter',
 'worst radius',
 'worst smoothness',
 'worst symmetry',
 'worst texture',
 'target',
 'prediction']

### 2 Drfit reports

#### 2.1 Data Drift reports between train and test/serving data 

In [35]:
target_drift_tests = Report(metrics=[DataDriftTable()])
target_drift_tests.run(current_data=bcancer_cur, reference_data=bcancer_ref, 
                       column_mapping=column_mapping)

/home/bala_33/Softwares/anaconda3/envs/tabular/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning:

divide by zero encountered in true_divide



In [37]:
data_drift_result = target_drift_tests.as_dict()

In [49]:
data_drift_result['metrics'][0]['result']['drift_by_columns']

{'target': {'column_name': 'target',
  'column_type': 'cat',
  'stattest_name': 'Z-test p_value',
  'drift_score': 0.13936735999235395,
  'drift_detected': False,
  'threshold': 0.05},
 'prediction': {'column_name': 'prediction',
  'column_type': 'cat',
  'stattest_name': 'chi-square p_value',
  'drift_score': 0.0,
  'drift_detected': True,
  'threshold': 0.05},
 'area error': {'column_name': 'area error',
  'column_type': 'num',
  'stattest_name': 'K-S p_value',
  'drift_score': 0.09514476355265436,
  'drift_detected': False,
  'threshold': 0.05},
 'compactness error': {'column_name': 'compactness error',
  'column_type': 'num',
  'stattest_name': 'K-S p_value',
  'drift_score': 0.821203623563264,
  'drift_detected': False,
  'threshold': 0.05},
 'concave points error': {'column_name': 'concave points error',
  'column_type': 'num',
  'stattest_name': 'K-S p_value',
  'drift_score': 0.43796584564429397,
  'drift_detected': False,
  'threshold': 0.05},
 'concavity error': {'column_name

In [62]:
drift_cols = []
for key, value in data_drift_result['metrics'][0]['result']['drift_by_columns'].items():
    if value['drift_detected']:
        drift_cols.append(value['column_name'])

In [63]:
drift_cols

['prediction']

#### 2.1 Test target drift in training and prediction

In [93]:
column_mapping = ColumnMapping()
column_mapping.target = 'target'
column_mapping.prediction = 'prediction'
column_mapping.datetime_features = datetime_columns
column_mapping.numerical_features = num_columns
column_mapping.categorical_features = cat_colummns
column_mapping.task = 'regression'

In [94]:
column_mapping

ColumnMapping(target='target', prediction='prediction', datetime='datetime', id=None, numerical_features=['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension'], categorical_features=None, datetime_features=None, target_names=None, task='regression', pos_label=1)

In [95]:
bcancer_ref.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension',
       'target', 'prediction'],
      dtype='object')

In [109]:
# dataset-level tests
# from evidently.test_suite import TestSuite
# from evidently.tests import *
# datadrift_tests = TestSuite(tests=[
#     TestTargetFeaturesCorrelations(),
# ])

# datadrift_tests.run(reference_data=bcancer_ref, current_data=bcancer_cur,
#                     column_mapping=column_mapping)
# datadrift_tests.as_dict()

In [111]:
target_drift_tests = Report(metrics=[TargetDriftPreset(),
                                    ])
target_drift_tests.run(current_data=bcancer_cur, 
                       reference_data=bcancer_ref,
                       column_mapping=column_mapping)

In [123]:
current_dict = target_drift_tests.as_dict()['metrics'][2]['result']['current']['pearson']['values']
current_dict['y1'] = target_drift_tests.as_dict()['metrics'][2]['result']['reference']['pearson']['values']['y']

In [136]:
import pandas as pd
corr_data = pd.DataFrame.from_dict(current_dict)
corr_data['corr_difference'] = abs(corr_data['y1']- corr_data['y'])
corr_data.loc[corr_data['corr_difference']>=0.05]['x'].tolist()

['area error',
 'compactness error',
 'mean fractal dimension',
 'mean smoothness',
 'smoothness error',
 'symmetry error',
 'worst fractal dimension']

In [132]:
corr_data

,x,y,y1,corr_difference
0,area error,-0.629923,-0.536494,0.093429
1,compactness error,-0.405445,-0.319074,0.086371
2,concave points error,-0.420380,-0.441772,0.021392
3,concavity error,-0.251480,-0.279231,0.027750
4,fractal dimension error,-0.144760,-0.150457,0.005696
5,mean area,-0.731386,-0.705677,0.025709
6,mean compactness,-0.616474,-0.581774,0.034701
7,mean concave points,-0.770958,-0.786056,0.015098
8,mean concavity,-0.690102,-0.717286,0.027184
9,mean fractal dimension,-0.090097,0.030341,0.120438


In [119]:
target_drift_tests.as_dict()['metrics'][2]['result']['reference']['pearson']['values']

{'x': ['area error',
  'compactness error',
  'concave points error',
  'concavity error',
  'fractal dimension error',
  'mean area',
  'mean compactness',
  'mean concave points',
  'mean concavity',
  'mean fractal dimension',
  'mean perimeter',
  'mean radius',
  'mean smoothness',
  'mean symmetry',
  'mean texture',
  'perimeter error',
  'radius error',
  'smoothness error',
  'symmetry error',
  'texture error',
  'worst area',
  'worst compactness',
  'worst concave points',
  'worst concavity',
  'worst fractal dimension',
  'worst perimeter',
  'worst radius',
  'worst smoothness',
  'worst symmetry',
  'worst texture'],
 'y': [-0.5364942055699388,
  -0.31907372024595804,
  -0.44177180873587457,
  -0.27923058420888536,
  -0.15045657482763905,
  -0.7056770549490284,
  -0.5817736690111365,
  -0.7860562605602806,
  -0.7172857698323594,
  0.030340938701770123,
  -0.7397258456723448,
  -0.7264215106735414,
  -0.3256843934748637,
  -0.33218185541865325,
  -0.3907686797731663,
  -